In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
import requests
import psycopg2
from io import BytesIO

In [2]:
load_dotenv()
SAM_PUBLIC_API_KEY = os.environ.get("SAM_PUBLIC_API_KEY")
S3_AWS_ACCESS_KEY_ID = os.environ.get("S3_AWS_ACCESS_KEY_ID")
S3_AWS_SECRET_ACCESS_KEY = os.environ.get("S3_AWS_SECRET_ACCESS_KEY")
S3_REGION_NAME = os.environ.get("S3_REGION_NAME")

POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [3]:
index_url = "https://www.census.gov/naics/2022NAICS/2022_NAICS_Index_File.xlsx"
description_url = "https://www.census.gov/naics/2022NAICS/2022_NAICS_Descriptions.xlsx"

In [4]:
res_index = requests.get

In [5]:
res_index = requests.get(index_url)

In [6]:
df_index = pd.read_excel(BytesIO(res_index.content))

In [7]:
df_index.head()

,NAICS22,INDEX ITEM DESCRIPTION
0,111110,"Soybean farming, field and seed production"
1,111120,"Canola farming, field and seed production"
2,111120,"Flaxseed farming, field and seed production"
3,111120,"Mustard seed farming, field and seed production"
4,111120,"Oilseed farming (except soybean), field and se..."


In [8]:
df_index.dtypes

NAICS22                   object
INDEX ITEM DESCRIPTION    object
dtype: object

In [9]:
mask = pd.to_numeric(df_index['NAICS22'], errors='coerce').isna()
df_index[mask]

,NAICS22,INDEX ITEM DESCRIPTION
20373,******,"Cards, publishing -- see specific product"
20374,******,"Clinics, medical -- see type"
20375,******,Consultants -- see specific activity
20376,******,Contractors -- see specific activity
20377,******,"Day camps, instructional -- see type of instru..."
20378,******,Dealers -- see type
20379,******,Farming -- see type
20380,******,Hospitals -- see type
20381,******,Instruction -- see type of training
20382,******,Laboratories -- see specific type


In [10]:
df_index = df_index.drop(df_index[mask].index)

In [11]:
df_index['NAICS22'] = df_index['NAICS22'].astype(int)

In [12]:
df_index.head()

,NAICS22,INDEX ITEM DESCRIPTION
0,111110,"Soybean farming, field and seed production"
1,111120,"Canola farming, field and seed production"
2,111120,"Flaxseed farming, field and seed production"
3,111120,"Mustard seed farming, field and seed production"
4,111120,"Oilseed farming (except soybean), field and se..."


In [13]:
df_index.drop(df_index[df_index['NAICS22'] < 111110].index, inplace=True)

In [14]:
df_index

,NAICS22,INDEX ITEM DESCRIPTION
0,111110,"Soybean farming, field and seed production"
1,111120,"Canola farming, field and seed production"
2,111120,"Flaxseed farming, field and seed production"
3,111120,"Mustard seed farming, field and seed production"
4,111120,"Oilseed farming (except soybean), field and se..."
...,...,...
20368,928120,Passport issuing services
20369,928120,Peace Corps
20370,928120,State Department
20371,928120,United Nations


In [15]:
res_desc = requests.get(description_url)
res_desc

<Response [200]>

In [16]:
df_desc = pd.read_excel(BytesIO(res_desc.content))

In [17]:
df_desc.head()

,Code,Title,Description
0,11,"Agriculture, Forestry, Fishing and HuntingT","The Sector as a Whole\n\nThe Agriculture, Fore..."
1,111,Crop ProductionT,Industries in the Crop Production subsector gr...
2,1111,Oilseed and Grain FarmingT,This industry group comprises establishments p...
3,11111,Soybean FarmingT,See industry description for 111110.
4,111110,Soybean Farming,This industry comprises establishments primari...


In [18]:
df_desc.rename(columns={'Code': 'NAICS22'}, inplace=True)

In [19]:
df_desc.head()

,NAICS22,Title,Description
0,11,"Agriculture, Forestry, Fishing and HuntingT","The Sector as a Whole\n\nThe Agriculture, Fore..."
1,111,Crop ProductionT,Industries in the Crop Production subsector gr...
2,1111,Oilseed and Grain FarmingT,This industry group comprises establishments p...
3,11111,Soybean FarmingT,See industry description for 111110.
4,111110,Soybean Farming,This industry comprises establishments primari...


In [20]:

df_desc['NAICS22'] = df_desc['NAICS22'].astype(int)

In [21]:
df_desc.drop(df_desc[df_desc['NAICS22'] < 111110].index, inplace=True)

TypeError: '<' not supported between instances of 'str' and 'int'